## 분류 모델링
## 로지스틱 회귀 모델 with TF-IDF

### 라이브러리 불러오기

In [16]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### 데이타 불러오기 - 텍스트 형태

In [17]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [18]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [19]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [20]:
reviews[:1]

['stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate worki

In [21]:
sentiments[:1]

[1]

### TF-IDF 벡터화
- min_df : 설정한 값보다 특정 토큰의 df 값이 적게 나오면 벡터화 과정에서 제거한다는 의미
- analyzer : 분석하기 위한 기준, 'char':문자 하나를 단위로 지정, 'word':단어 하나를 단위로 지정
- sublinear_tf : 문서의 단어 빈도수(term frequency)에 대한 스무딩(smoothing) 여부를 설정하는 값
- ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 지정하는 인자
- max_features : 각 벡터의 최대 길이, 특징의 길이를 설정

- TfidVectorizer 객체를 생성 --> fit_transform() 함수를 사용해 전체 문장에 대한 특징 벡터 데이타 X를 생성
- y - 라벨(정답) 데이타

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [23]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 17862871 stored elements in Compressed Sparse Row format>

In [25]:
features = vectorizer.get_feature_names()
features[:10]

[' ', ' a', ' aa', ' ab', ' ac', ' ad', ' ae', ' af', ' ag', ' ah']

### 학습 데이타와 테스트 데이타로 분리
- 사이킷런의 train_test_split 함수 이용
- 학습데이타:테스트데이타 = 80:20

In [29]:
from sklearn.model_selection import train_test_split
# RANDOM_SEED = 42
# TEST_SPLIT = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=TEST_SPLIT, random_state=RANDOM_SEED)

### 로지스틱 회귀모델 생성 
- class_weight='balanced': 각 레벨에 대해 균형있게 학습할 수 있게 설정

In [30]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced') 

### 로지스틱 회귀모델 학습: fit()

In [31]:
lgs.fit(X_train, y_train) 

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 모델 예측 및 모델 평가 
- 테스트 데이타로 모델 예측 및 평가

In [32]:
predicted = lgs.predict(X_test)

In [33]:
print("Accuracy: %f" % lgs.score(X_test, y_test))

Accuracy: 0.859800


- 해석) 
- 정확도가 85% 정도 나옴

### 생성된 로지스틱 회귀모델을 이용하여 평가 데이타 결과를 예측

- 평가 데이타 파일 불러오기: 텍스트 형태 파일

In [35]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [39]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


- 평가 데이타도 학습 데이타와 동일한 TF-IDF 벡터화 진행
- 주의) fit을 호출하지 않고 transform만 호출
    - fit은 학습 데이타에 맞게 설정, 그 설정에 맞게 평가 데이타도 변환해야 한다.

In [36]:
testDataVecs = vectorizer.transform(test_data['review'])

- 생성된 로지스틱회귀모델을 이용하여 예측

In [37]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


- 예측한 결과값을 데이타프레임형태로 만들어 csv 파일로 저장
- 각 데이타의 고유값 id와 결과값으로 구성
- 캐글에 csv 파일 제출한 후 정확도 확인

In [38]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)